#### Prompt Template and Chaining - Hugging Face

In [ ]:
# %pip install langchain huggingface_hub

In [ ]:
from langchain_core.prompts import PromptTemplate 
from langchain.llms import HuggingFaceEndpoint

# Set your Hugging Face API token
huggingfacehub_api_token = ""

# Create a prompt template from the template string
template = "You are an artificial intelligence assistant, answer the question. {question}"
prompt_template = PromptTemplate(template = template, input_variables = ["question"])

# Create a chain to integrate the prompt template and LLM
llm = HuggingFaceEndpoint(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token)
llm_chain = prompt_template | llm

question = "How does LangChain make LLM application development easier?"
print(llm_chain.invoke({"question": question})) 

C:\Users\kyith\AppData\Local\Temp\ipykernel_44876\2918683206.py:12: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token)
C:\Users\kyith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kyith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Loc


LangChain provides a suite of LLM-specific libraries and tools, which can significantly reduce the time and effort required to develop and maintain LLM applications. These libraries cover various aspects of LLM development, including document generation, model compilation, and rule-based programming. By using these libraries, LLM developers can focus on the core tasks of their applications and avoid manual coding, ultimately resulting in faster development times and higher quality applications.


#### Chat Prompt Template - OpenAI 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Define an OpenAI chat model
llm = ChatOpenAI(model="gpt-4o-mini", api_key='<OPENAI_API_TOKEN>')		

# Create a chat prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Respond to question: {question}")
    ]
)

# Chain the prompt template and model, and invoke the chain
llm_chain = prompt_template | llm
response = llm_chain.invoke({"question": "How can I retain learning?"})
print(response.content)

#### Few Shot Prompting

In [ ]:
# Create the examples list of dicts
examples = [
    {
        "question": "How many DataCamp courses has Jack completed?",
        "answer": "36"
    },
    {
        "question": "How much XP does Jack have on DataCamp?",
        "answer": "284,320XP"
    },
    {
        "question": "What technology does Jack learn about most on DataCamp?",
        "answer": "Python"
    },
]

In [7]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Complete the prompt for formatting answers
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

# Create the few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

# Invoke the prompt template
prompt = prompt_template.invoke({"input": "What is Jack's favorite technology on DataCamp?"})
print(prompt.text)

Question: How many DataCamp courses has Jack completed?
36

Question: How much XP does Jack have on DataCamp?
284,320XP

Question: What technology does Jack learn about most on DataCamp?
Python

Question: What is Jack's favorite technology on DataCamp?


In [ ]:
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

# Create an OpenAI chat LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key='<OPENAI_API_TOKEN>')

# Create and invoke the chain
llm_chain = prompt_template | llm
print(llm_chain.invoke({"input": "What is Jack's favorite technology on DataCamp?"}))

### Chain and Agents

#### Sequential Chains

In [9]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables = ["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "play golf"}))

text='I want to learn how to play golf. Can you suggest how I can learn this step-by-step?'


In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "play golf"}))

In [1]:
# %pip install langgraph == 0.066
%pip install numexp

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement numexp (from versions: none)
ERROR: No matching distribution found for numexp


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key='OpenAI_API_TOKEN') 

tools = load_tools(["llm-math"], llm  = llm)
agent = create_react_agent(tools = tools) # Create an agent with the tools

messages = agent.invoke({"messages:"[("human","What is the square root of 101?")]}) # Invoke the agent
print(messages)
print(messages[messages][-1].content)

Exercises:

In [ ]:
# Define the tools
tools = load_tools(["wikipedia"])

# Define the agent
agent = create_react_agent(llm, tools)

# Invoke the agent
response = agent.invoke({"messages": [("human", "How many people live in New York City?")]})
print(response["messages"][-1].content)

#### Custom tools for agent

In [ ]:
# Custom tools for agent

# tool formats
from langchain_community.agent_toolkits.load_tools import load_tools

tools = load_tools(["llm-math", "llm-wikipedia"], llm = llm)
print(tools[0].name) #get the name of the first tool
print(tools[0].format) #get the format of the first tool
print(tools[0].description) #get the description of the first tool

print(tools[0].return_direction) #get the return direction of the first tool

In [4]:
# want to define a Python function to generate a financial report for a company. 
# It takes three arguments: the company_name, revenue, and expenses, 
# and outputs a string containing the net_income.

def financial_report(company_name:str, revenue:int, expenses:int) -> str:
    '''Generate a financial report for a company that calculates the net income.'''
    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

# calling the function
# print(financial_report("ABC Corp", 100000, 75000))
print(financial_report(company_name="Lemonade stand", revenue=100, expenses=50))

Financial Report for Lemonade stand:
Revenue: $100
Expenses: $50
Net Income: $50



In [6]:
# from functions to tools
from langchain_core.tools import tool

@tool
def financial_report(company_name:str, revenue:int, expenses:int) -> str:
    '''Generate a financial report for a company that calculates the net income.'''
    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

print(financial_report.name) #get the name of the tool
print(financial_report.description) #get the desctiption of the tool
print(financial_report.return_direct) #get the return direction of the tool
print(financial_report.args) #get the arguments of the tool

financial_report
Generate a financial report for a company that calculates the net income.
False
{'company_name': {'title': 'Company Name', 'type': 'string'}, 'revenue': {'title': 'Revenue', 'type': 'integer'}, 'expenses': {'title': 'Expenses', 'type': 'integer'}}


In [ ]:
# use ReAct agent
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key, temperature=0)
agent = create_react_agent(llm, [financial_report])

messages = agent.invoke({"messages": [("human", "Generate a financial report for ABC Corp with revenue of $100,000 and expenses of $75,000.")]})
print(messages)

### RAG